In [ ]:
# 3) ODEs + truth simulators (FN / LV (log) / Hes1)

# ---------- Dynamics ----------
def FN(y, t, a, b, c):
    V, R = y
    dVdt = c * (V - V**3/3.0 + R)
    dRdt = -1.0/c * (V - a + b*R)
    return (dVdt, dRdt)

def LV_log(y, t, a, b, c, d):
    # state in log-space
    x1, x2 = np.exp(y)
    dx1dt = a*x1 - b*x1*x2
    dx2dt = c*x1*x2 - d*x2
    return [dx1dt/x1, dx2dt/x2]  # chain rule

def Hes1(y, t, a, b, c, d, e, f, g):
    P, M, H = y
    dPdt = -a*P*H + b*M - c*P
    dMdt = -d*M + e/(1 + P**2)
    dHdt = -a*P*H + f/(1 + P**2) - g*H
    return (dPdt, dMdt, dHdt)

# ---------- Truth simulators ----------
def simulate_FN():
    a, b, c = 0.2, 0.2, 3.0
    V0, R0 = -1.0, 1.0
    t = np.linspace(0, 40, 1281)
    X = odeint(FN, (V0, R0), t, args=(a, b, c))
    return t, X

def simulate_LV_log():
    a, b, c, d = 1.5, 1.0, 1.0, 3.0
    x1_0, x2_0 = 5.0, 0.2
    y0 = np.log([x1_0, x2_0])
    t = np.linspace(0, 12, 321)
    Y = odeint(LV_log, y0, t, args=(a, b, c, d))  # log-state
    X = np.column_stack([np.exp(Y[:,0]), np.exp(Y[:,1])])  # back to original scale
    return t, X

def simulate_Hes1():
    a, b, c, d, e, f, g = 0.022, 0.3, 0.031, 0.028, 0.5, 20.0, 0.3
    P0, M0, H0 = 1.438575, 2.037488, 17.90385
    t = np.linspace(0, 640, 1281)
    X = odeint(Hes1, (P0, M0, H0), t, args=(a, b, c, d, e, f, g))
    return t, X

In [ ]:
# 4) Observation maker (same style you used)

def make_observations(t, X, no_train, noise, seed=0):
    """
    t: (T,), X: (T,D)
    noise: list-like of length D (std dev per component)
    returns: list of arrays [(n_i, 2) ...] with columns (t, y_obs)
    and the last observation time (fit boundary).
    """
    np.random.seed(seed)
    T, D = X.shape
    # observe first half of the horizon unless you change obs_idx
    obs_idx = np.linspace(0, (T-1)//2, no_train).astype(int)
    obs = []
    for d in range(D):
        tobs = t[obs_idx].copy()
        yobs = X[obs_idx, d].copy() + np.random.normal(0, noise[d], size=no_train)
        obs.append(np.c_[tobs, yobs])
    t_end_fit = max(o[:,0].max() for o in obs)
    return obs, t_end_fit